# Download datasets (Necessario chave api kaggle)

- Crie uma conta em [Kaggle](https://www.kaggle.com/)
- Settings
- Api
- Create New Token


In [ ]:
! pip install -q opendatasets

In [ ]:
import os
import re
from datetime import datetime

import numpy as np
import opendatasets as od
import pandas as pd
from IPython.display import clear_output

In [ ]:
clear_output()


od.download(
    "https://www.kaggle.com/datasets/josephcheng123456/olympic-historical-dataset-from-olympediaorg"
)

# Codigos


## imports


In [ ]:
def conv_data(valor: str) -> str | None:
    try:
        data_datetime = datetime.strptime(valor, "%d %B %Y")
        return data_datetime
    except:  # noqa: E722
        return None


def get_year(valor: str) -> str | None:
    try:
        year = re.findall(r"\d{4}", valor)[0]
        return np.int64(year)
    except:  # noqa: E722
        return pd.NA

## athlete_bio


In [ ]:
def conv_data(valor: str) -> str | None:  # noqa: F811
    try:
        data_datetime = datetime.strptime(valor, "%d %B %Y")
        return data_datetime
    except:  # noqa: E722
        return None


def get_year(valor: str) -> str | None:  # noqa: F811
    try:
        year = re.findall(r"\d{4}", valor)[0]
        return np.int64(year)
    except:  # noqa: E722
        return pd.NA


dtypes = {
    "athlete_id": str,
    "name": str,
    "sex": str,
    "born": str,
    "height": float,
    "country": str,
    "country_noc": str,
    "description": str,
    "special_notes": str,
}

df = pd.read_csv(
    "/content/olympic-historical-dataset-from-olympediaorg/Olympic_Athlete_Bio.csv",
    dtype=dtypes,
)

df.weight = pd.to_numeric(df.weight, errors="coerce")

tdata = df["born"].apply(conv_data)
anos = df["born"].apply(get_year)

tdata = tdata.dt.strftime("%Y-%m-%d")
df.born = tdata

df["year_born"] = anos

os.makedirs("output", exist_ok=True)

df.to_csv("output/athlete_bio.csv", index=False)

## athlete_event_result


In [ ]:
df = pd.read_csv(
    "/content/olympic-historical-dataset-from-olympediaorg/Olympic_Athlete_Event_Results.csv",
    dtype=str,
)

dicionario = {
    "DNF": "Did not finish",
    "DNS": "Did not start",
    "DQ": "Disqualified",
    "AC": "Also competed, no definite place known or possibly given",
    "NH": "No height",
    "NM": "No mark",
    "NP": "Not placed, no definite place given",
    "NVL": "No valid lift",
    "MNK": "Mark not known",
    "TNK": "Time not known",
    "DNQ": "Did not qualify",
    "HC": "Hors concours (art)",
}

os.makedirs("output", exist_ok=True)

mask = df.pos.isin(dicionario.keys())

for place, replace in dicionario.items():
    df.loc[mask, "pos"] = df.loc[mask, "pos"].str.replace(place, replace)

df.to_csv("/content/output/athlete_event_result.csv", index=False)

## game_medal_tally


In [ ]:
df = pd.read_csv(
    "/content/olympic-historical-dataset-from-olympediaorg/Olympic_Games_Medal_Tally.csv",
    dtype={"edition_id": str},
)

os.makedirs("output", exist_ok=True)

df.to_csv("output/game_medal_tally.csv", index=False)

## result


In [ ]:
df = pd.read_csv(
    "/content/olympic-historical-dataset-from-olympediaorg/Olympic_Results.csv",
    dtype=str,
)

os.makedirs("output", exist_ok=True)

df.to_csv("output/result.csv", index=False)

## country


In [ ]:
df = pd.read_csv(
    "/content/olympic-historical-dataset-from-olympediaorg/Olympics_Country.csv",
    dtype=str,
)

os.makedirs("output", exist_ok=True)

df.to_csv("output/country.csv", index=False)

## game


In [ ]:
df = pd.read_csv(
    "/content/olympic-historical-dataset-from-olympediaorg/Olympics_Games.csv",
    dtype={"edition_id": str},
)

start_tdata = (
    df.start_date.str.replace(" 2021", "") + " " + df.year.astype(str)
)  # Uma das datas já se encontra com o ano de 2021, necessario remoção
end_tdata = df.end_date.str.replace(" 2021", "") + " " + df.year.astype(str)

start_tdata = start_tdata.apply(conv_data)
end_tdata = end_tdata.apply(conv_data)

df["start_date"] = start_tdata
df["end_date"] = end_tdata

os.makedirs("output", exist_ok=True)

df.to_csv("output/game.csv", index=False)